In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf 
import os,sys 
sys.setrecursionlimit(15000)
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD,Adam
from keras.utils import np_utils
from keras.callbacks import *
from keras.layers import *
from sklearn.model_selection import train_test_split
from keras.models import Sequential ,Model,load_model


In [ ]:
map_characters = {0: 'abraham_grampa_simpson', 1: 'apu_nahasapeemapetilon', 2: 'bart_simpson', 
        3: 'charles_montgomery_burns', 4: 'chief_wiggum', 5: 'comic_book_guy', 6: 'edna_krabappel', 
        7: 'homer_simpson', 8: 'kent_brockman', 9: 'krusty_the_clown', 10: 'lenny_leonard', 11: 'lisa_simpson', 
        12: 'marge_simpson', 13: 'mayor_quimby', 14: 'milhouse_van_houten', 15: 'moe_szyslak', 
        16: 'ned_flanders', 17: 'nelson_muntz', 18: 'principal_skinner', 19: 'sideshow_bob'}


pic_size = 64
batch_size = 64
epochs = 500
num_classes = len(map_characters)

images = []
labels = []
name  = []



In [ ]:
def read_images_labels(path,i):
  for file in os.listdir(path):
    #pics_path = os.path.join("/content/drive/MyDrive/CharacterML_Hw/train/characters-20")
    abs_path =os.path.abspath(os.path.join(path,file))
    if os.path.isdir(abs_path):
      i+=1
      temp = os.path.split(abs_path)[-1]
      name.append(temp)
      read_images_labels(abs_path,i)
      amount = int(len(os.listdir(path)))
      sys.stdout.write('\r'+'>'*(i)+' '*(amount-i)+'[%s%%]'%(i*100/amount)+temp)
          
    else:
        if file.endswith('.jpg'):
          image = cv2.resize(cv2.imread(abs_path),(64,64))
          images.append(image)
          labels.append(i-1)
  return images , labels ,name      

In [ ]:
def read_main(path):
  global images
  image,labels,name = read_images_labels(path,i=0)
  images = np.array(images,dtype=np.float32)/255
  labels = np_utils.to_categorical(labels,num_classes=20)
  np.savetxt('name.txt',name,delimiter=' ',fmt='%s')
  return images ,labels

images,labels = read_main('/content/drive/MyDrive/CharacterML_Hw/train/characters-20')
x_train,x_test,y_train,y_test = train_test_split(images,labels ,test_size=0.1)

>>>>>>>>>>>>>>>>>>>>[100.0%]moe_szyslak

In [ ]:
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu', input_shape = x_train.shape[1:]))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Conv2D(filters = 86, kernel_size = (3,3),padding = 'Same', 
                     activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(20, activation = "softmax"))
    model.summary()
    model.compile(optimizer = 'Adam' , loss = "categorical_crossentropy", metrics=["accuracy"])
    
    datagen = ImageDataGenerator(zoom_range=0.1,width_shift_range=0.05,height_shift_range=0.05,horizontal_flip=True)
    datagen.fit(x_train)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
dropout_16 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 64)       

In [ ]:
import h5py
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=20)


In [ ]:
file_name = str(epochs)+'_'+str(batch_size)
fig = model.fit_generator(datagen.flow(x_train,y_train,batch_size=batch_size),steps_per_epoch=epochs,epochs=epochs,validation_data=(x_test,y_test),verbose = 1,callbacks=early_stopping)
model.save('/content/drive/MyDrive/CharacterML_Hw/train/model'+file_name+'.h5')
score = model.evaluate(x_test,y_test,verbose=1)
print(score)

Epoch 1/500
500/500 [==============================] - 36s 72ms/step - loss: 1.9158 - accuracy: 0.4772 - val_loss: 1.1848 - val_accuracy: 0.6399
Epoch 2/500
500/500 [==============================] - 36s 72ms/step - loss: 1.0541 - accuracy: 0.6867 - val_loss: 0.6793 - val_accuracy: 0.7964
Epoch 3/500
500/500 [==============================] - 36s 72ms/step - loss: 0.7638 - accuracy: 0.7729 - val_loss: 0.6627 - val_accuracy: 0.8102
Epoch 4/500
500/500 [==============================] - 36s 72ms/step - loss: 0.6195 - accuracy: 0.8167 - val_loss: 0.4770 - val_accuracy: 0.8568
Epoch 5/500
500/500 [==============================] - 36s 72ms/step - loss: 0.5329 - accuracy: 0.8415 - val_loss: 0.3727 - val_accuracy: 0.8934
Epoch 6/500
500/500 [==============================] - 36s 71ms/step - loss: 0.4645 - accuracy: 0.8609 - val_loss: 0.3903 - val_accuracy: 0.8806
Epoch 7/500
500/500 [==============================] - 35s 71ms/step - loss: 0.4219 - accuracy: 0.8722 - val_loss: 0.4632 - val_ac

KeyboardInterrupt: ignored

In [ ]:
def read_images(path):
  images=[]
  for i in range(990):
    image=cv2.resize(cv2.imread(path+str(i+1)+'.jpg'),(64,64))
    images.append(image)
  images = np.array(images,dtype = np.float32)/255
  return images
  

In [ ]:
def transform(listdir,label,lenSIZE):
  label_str=[]
  for i in range (lenSIZE):
    temp = listdir[label[i]]
    label_str.append(temp)
  return label_str

images = read_images('/content/drive/MyDrive/CharacterML_Hw/test/test/')
model = load_model('/content/drive/MyDrive/CharacterML_Hw/train/model10_128.h5')

predict = model.predict_classes(images, verbose=1)
print(predict)
label_str=transform(np.loadtxt('name.txt',dtype='str'),predict,images.shape[0])


31/31 [==============================] - 0s 3ms/step
[16  3 17 17  7 11 18 19 12  4 18 13 15 11  8 18 13  7  4  3  7  5  7  8
  5  0 13 11  4 18  4  8  1  8  0  8 12  2  2 18  1  2 10 17  5  6 12  1
 15 14 10  8  4  6 11 13  5  7 16  4 19  9 19  4  3 17  4 10  6 19  7  2
  6  9 15 19 11 18 10  4  5 14  6 13  8 11  7  7 14  5 16 10 14 15 15 15
  3  3  7  1  7 17  5 19  9 14 17  7 17 12 13 15  2 15 18  4  2  6  9 17
  1  1 10 10 11 14 12 10 11  1 14 19 18  6  8  5  2  3  5 12  8 15  2  1
  2  1 11  4 15  3 18  0 10  1 13  1  4 19  8 13 18  1  1  5  9 15 12 11
 14  3 14  1  5  1 14 10 14 15  1 10  4  2 15  1  9  3 15  1 11 10 17 18
 12  7 18 16  1 17 19  8 14 15  0  6 10 10 10 16  8  4 15 15 17  1  2  1
 13  0  9 12  6 10 12 17 18  8  6  8 19  5  5  1 18 12  0 17  6  4  9  9
 13  1  6 11  1 18 12  6  2  3 19  8  6  0  2  1 15 14  6  7 11  3  5  6
  9  8  0 10 18  4  8  0  4  1  1  9  5 19 14  5 14  1  6 13  1 16  7  0
 16  2 12 16 15  3  7  2 12 13  8 19  6 17 16 13  6  7  6 11  7 16  9  

In [ ]:
df = pd.DataFrame({'character':label_str})
df.index = np.arange(1,len(df)+1)
df.index.names = ['id']
df.to_csv('/content/drive/MyDrive/CharacterML_Hw/'+'test'+'.csv')